In [4]:
import os
import getpass
from typing import TypedDict, Annotated, List
import operator
from langchain_core.messages import BaseMessage
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

# 1. SETUP API KEY (Crucial Fix)
if not os.environ.get("GROQ_API_KEY"):
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter Groq API Key: ")

# 2. INITIALIZE LLM
llm = ChatGroq(model="llama-3.1-70b-versatile", temperature=0)

class AgentState(TypedDict):
    ticker: str
    messages: Annotated[List[BaseMessage], operator.add]
    technical_summary: str
    fundamental_summary: str
    final_report: str

print("✅ Setup Complete: Keys & State Ready.")

✅ Setup Complete: Keys & State Ready.


In [5]:
import yfinance as yf
from langchain_core.tools import tool

@tool
def get_trade_signals(ticker: str):
    """Calculates Support, Resistance, and Stop Loss."""
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(period="1y")
        if hist.empty: return "Error: No Data found."

        hist['Support'] = hist['Low'].rolling(window=60).min()
        hist['Resistance'] = hist['High'].rolling(window=60).max()
        hist['SMA_20'] = hist['Close'].rolling(window=20).mean()
        hist['STD_20'] = hist['Close'].rolling(window=20).std()
        hist['BB_Lower'] = hist['SMA_20'] - (hist['STD_20'] * 2)
        
        curr = hist.iloc[-1]
        price = curr['Close']
        stop_loss = curr['Support'] * 0.97
        
        return (
            f"Price: {price:.2f}\n"
            f"Support: {curr['Support']:.2f}\n"
            f"Resistance: {curr['Resistance']:.2f}\n"
            f"Stop Loss: {stop_loss:.2f}\n"
        )
    except Exception as e:
        return f"Error: {e}"

print("✅ Tools Re-loaded.")

✅ Tools Re-loaded.


In [6]:
# --- 1. DEFINE NODES ---
def quant_node(state):
    ticker = state['ticker']
    print(f"🔹 Quant Agent: Analyzing {ticker}...")
    
    # Use the tool we defined in Cell 2
    raw_data = get_trade_signals.invoke(ticker)
    
    res = llm.invoke(f"Analyze this data and give a BUY/SELL signal with Stop Loss: {raw_data}")
    return {"technical_summary": res.content}

def research_node(state):
    ticker = state['ticker']
    print(f"🔹 Research Agent: Searching PDF for {ticker}...")
    
    # Connect to the DB we built in Notebook 2
    context = "No PDF data found."
    try:
        embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
        db = Chroma(persist_directory="./chroma_db", embedding_function=embedding)
        retriever = db.as_retriever(search_kwargs={"k": 2})
        docs = retriever.invoke(f"{ticker} risks")
        if docs:
            context = "\n".join([d.page_content for d in docs])
    except:
        pass
    
    res = llm.invoke(f"Summarize financial risks based on this text: {context}")
    return {"fundamental_summary": res.content}

def manager_node(state):
    print("🔹 Manager: Aggregating...")
    return {
        "final_report": f"### FINAL REPORT\n\n**Techs:**\n{state['technical_summary']}\n\n**Funds:**\n{state['fundamental_summary']}"
    }

# --- 2. BUILD GRAPH ---
workflow = StateGraph(AgentState)

workflow.add_node("quant", quant_node)
workflow.add_node("research", research_node)
workflow.add_node("manager", manager_node)

workflow.add_edge(START, "quant")
workflow.add_edge(START, "research")
workflow.add_edge("quant", "manager")
workflow.add_edge("research", "manager")
workflow.add_edge("manager", END)

# MemorySaver requires no extra imports now as they are in Cell 1
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

print("✅ Graph Compiled Successfully.")

✅ Graph Compiled Successfully.
